In [42]:
import numpy as np
import random


class Machine:
    
    def __init__(self, vulnerable, cost):
        self.vulnerable = vulnerable
        self.cost = cost
        
    def changeVulnerable(self, vulnerable):
        self.vulnerable = vulnerable
    
    def changeCost(self, cost):
        self.cost = cost
        
    def pr(self):
        print(self.vulnerable, self.cost)

        
def printAllvalues(machineValues, vulnerableMachineValues, importantMachineValues):
    print('machine values = ', machineValues, 'vulnerable machine values = ', vulnerableMachineValues, 'important machine values =', importantMachineValues)

def GetMachineValues():
    machineValues = random.randint(1, 15)
    return machineValues

def GetVulnerableMachineValues(machineValues):
    vulnerableMachineValues = random.randint(1, machineValues)
    return vulnerableMachineValues

def GetImportantMachineValues(machineValues):
    importantMachineValues = random.randint(1, machineValues)
    return importantMachineValues

def GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues):
    #пока что сделаем цену cost фиксированной
    #дальше придумаем фукнцию для нее
    machineArray = []
    for i in range(machineValues):
        tmp = Machine(False, 10)
        machineArray.append(tmp)
    #здесь баг - может 2 и более раз сгенерироваться
    #одинаковое число и поэтому кол-во машин будет неточным
    for i in range(0, vulnerableMachineValues):
        j = random.randint(0, machineValues - 1)
        print('j = ', j)
        machineArray[j].changeVulnerable(True)
    return machineArray
#возвращает число уязвимых машин, так как из-за бага
#прошлое число уязвимых != новому
def GetVulnerableMachine(machineValues, machineArray):
    vulnerableMachine = 0
    for i in range(machineValues):
        if machineArray[i].vulnerable == True:
            vulnerableMachine += 1
    return vulnerableMachine

def CostFunction(s, a, Ei):
    return -(s * Ei *Ei)/
    

#def AttackerStrategyProportional(mv):
    

#def AttackerStrategyHigh(mv):
    

#def DefenderStrategyNothing(mv):
    

#def DefenderStrategyProportional(mv):
    
    
#def DefenderStrategyProportional(mv):
    
    
#def UtilityAttacker():
    

#def UtilityDefender():
    
    
machineValues = GetMachineValues()
vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
importantMachineValues = GetImportantMachineValues(machineValues)
machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
printAllvalues(machineValues, vulnerableMachineValues, importantMachineValues)
for i in range(machineValues):
    machineArray[i].pr()
vulnerableMachine = GetVulnerableMachine(machineValues, machineArray)
print(vulnerableMachine)
#for s in (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1):
#    for a in range(1, 11, 1):
#        for strategyA in (1, 2):
#            for strategyD in (1, 2, 3):
#                for Ei in (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9):


j =  7
j =  3
machine values =  9 vulnerable machine values =  2 important machine values = 5
False 10
False 10
False 10
True 10
False 10
False 10
False 10
True 10
False 10
2


In [39]:
import random

machineValues = random.randint(1, 100)
vulnerableMachineValues = random.randint(1, machineValues)
print(machineValues, vulnerableMachineValues)
counter = 0
for i in range(1, vulnerableMachineValues+1):
    tmp = random.randint(1, machineValues)
    counter += 1
    print(tmp)
print('c =', counter)

80 67
5
15
70
33
56
7
43
65
33
71
60
44
42
65
19
68
33
73
1
6
66
63
12
10
67
67
28
62
12
6
70
23
30
58
19
23
73
51
23
5
7
63
71
38
72
16
10
40
52
47
53
25
78
45
18
49
23
23
51
24
31
78
57
1
56
64
13
c = 67


In [15]:
import sys

class Machine:
    
    def __init__(self, v, cost):
        self.v = v
        self.cost = cost
        
    def change(self, v, cost):
        self.v = v
        self.cost = cost
        
    def pr(self):
        print(self.v, self.cost)

        
aM = []
cost = 13
for i in range(10):
    tmp = Machine(False, cost)
    aM.append(tmp)
for i in range(10):
    aM[i].pr()
print('_____')
for i in range(10):
    if i == 0 or i == 3 or i == 5 or i == 9:
        aM[i].change(True, 16)
    aM[i].pr()

#machine1 = Machine(True, 45)
#machine1.pr()
#print(machine1.__sizeof__())
#print(sys.getsizeof(machine1))
#print('__________')
#a = (True, 45)
#print(a.__sizeof__())
#print(sys.getsizeof(a))

False 13
False 13
False 13
False 13
False 13
False 13
False 13
False 13
False 13
False 13
_____
True 16
False 13
False 13
True 16
False 13
True 16
False 13
False 13
False 13
True 16


In [18]:
def a(b):
    return b

print(a(4))

4
